In [2]:
import QUANTAXIS as QA
import pandas as pd
import numpy as np
from pprint import  pprint as print
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
import keras
import sklearn
from sklearn import model_selection
from sklearn import preprocessing
import time
import random
from functools import wraps
import pysnooper


def fn_timer(function):
  @wraps(function)
  def function_timer(*args, **kwargs):
    t0 = time.time()
    result = function(*args, **kwargs)
    t1 = time.time()
    print ("Total time running %s: %s seconds" %
        (function.__name__, str(t1-t0)))
    return result
  return function_timer
#stockCode=QA.QA_fetch_stock_list()['code'].tolist()
stockCode = ['601318','600351']
QAdata = QA.QA_fetch_stock_day_adv(stockCode, '2018-03-20', '2018-08-23')

@pysnooper.snoop()
def max_min_close_n_days(data, n=30):
    '''
    自定义指标，计算当日（不含）到n天后收盘价最大值、最小值、最大值上涨比率、最小值下跌比率
    返回：
    n天内最高价:n_days_max
    n天内最低价:n_days_min
    n天内最高价较最近一个收盘价涨跌幅:n_days_max_radio
    n天内最低价较最近一个收盘价涨跌幅:n_days_min_radio
    QAdata = QA.QA_fetch_stock_day_adv(stockCode, '2018-03-20', '2018-08-23')
    max_min_close_n_days_radio = QAdata.add_func(max_min_close_n_days)
    '''
    #print(data.index[0][1])
    dataLen=len(data)
    n_days_max=np.full(dataLen, np.nan)
    n_days_min=np.full(dataLen, np.nan)
    n_days_max_radio=np.full(dataLen, np.nan)
    n_days_min_radio=np.full(dataLen, np.nan)
    
    n_days_2D=np.empty([dataLen,n])
    if dataLen>30:
        for i in range(n):
            n_days_temp=np.concatenate([data['close'].iloc[i+1:],np.empty(i+1)])
            '''if data.index[0][1]=='000526' or data.index[0][1]=='000526' :
                print('...')
                print(data)
                print(n_days_temp)
                print(n_days_2D[:,i])'''
            n_days_2D[:,i]=n_days_temp
            #print(n_days_temp)

        #print(n_days_2D[0,:])
        #print(n_days_max_temp)

        n_days_max=np.amax(n_days_2D,axis=1)
        n_days_min=np.amin(n_days_2D,axis=1)
        n_days_max_radio=(n_days_max-data['close'])/data['close']
        n_days_min_radio=(n_days_min-data['close'])/data['close']

        n_days_max[dataLen-n:dataLen]=None
        n_days_min[dataLen-n:dataLen]=None
        n_days_max_radio[dataLen-n:dataLen]=None
        n_days_min_radio[dataLen-n:dataLen]=None
    max_min_close_n_days=pd.DataFrame({'n_days_max':n_days_max,'n_days_min':n_days_min,
                                       'n_days_max_radio':n_days_max_radio,'n_days_min_radio':n_days_min_radio})
    max_min_close_n_days.index=data.index
    return max_min_close_n_days.dropna()

@fn_timer
def rate2oneshot(radio,m=30):
    '''
    传入参数为一个变化幅度的时间序列
    这个函数返回一个m时序后的one-hot时间序列
    
    变化幅度30%以上为[0,0,0,1]
    变化幅度10%-30%为[0,0,1,0]
    变化幅度 0%-10%为[0,1,0,0]
    变化幅度 0%以下为[1,0,0,0]
    
    '''
    
    radio=radio.copy()
    radio[radio>0.3]=3
    radio[(radio>0.1)&(radio<=0.3)]=2
    radio[(radio>0)&(radio<=0.1)]=1
    radio[radio<=0]=0
    radio.shape=(-1,1)
    
    enc = preprocessing.OneHotEncoder(categories='auto')
    enc.fit([[0],[1],[2],[3]])
    radio_onehot=enc.transform(radio).toarray()
    return radio_onehot
    '''
    for code in radio.index.levels[1]:
        arr=radio.loc[(slice(None),code),'n_days_max_radio'].values
        ser=pd.Series(enc.transform(arr.reshape(-1,1)).toarray().tolist())
        print(radio.loc[(slice(None),code),'n_days_max_radio'])
        radio.loc[(slice(None),code),'n_days_max_radio']=ser
        print(radio.loc[(slice(None),code),'n_days_max_radio'])
    r=pd.concat([radio,data.data],axis=1)
    '''

dataAndRadio = QAdata.add_func(max_min_close_n_days)
dataAndRadio=dataAndRadio.dropna()
rate_onehot=rate2oneshot(dataAndRadio['n_days_max_radio'].values)
y_df_rate_onehot=pd.DataFrame(rate_onehot,index=dataAndRadio.index)
y_df_rate_onehot.insert(0,'n_days_max_radio',dataAndRadio['n_days_max_radio'])


'Total time running rate2oneshot: 0.0 seconds'


In [2]:
from sklearn import preprocessing
def func_standardization(df):
    #对数据去中心化，使符合正态分布
    
    #部分不同列的数据具有相关性，需要统一标准化
    assemble_scale_column=['open','close','high','low']
    ar = preprocessing.scale(np.ravel(df[assemble_scale_column]))
    
    #对所有数据标准化后替换，对特殊列标准化后覆盖替换
    df[df.columns]=preprocessing.scale(df)
    df[assemble_scale_column]=ar.reshape([len(df),-1])
    
    return df

df=QAdata.data
x_df=df.groupby('code').apply(func_standardization)
#pd.concat([x_df,df],axis=1).loc[(slice(None),'601318'),:]

d:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:172: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [13]:
def make_timeseries_data(x_df,n=20):
    if(n>len(x_df)):
        rasie 
    index = pd.MultiIndex.from_tuples([], names=['date', 'code'])
    x = pd.DataFrame(columns=x_df.columns,index=index)

    for code in x_df.index.levels[1]:
        code_data=x_df.loc[(slice(None),code),:]
        for i in range(len(code_data)):
            x=pd.concat([x,x_df.iloc[i:i+n,:]],axis=0)
    
        
        
def standardization(data,groupby='code'):
    for code in data.index.levels[1]:
        pass



Index(['600351', '601318'], dtype='object', name='code')

In [20]:
c=[[1,2],[3,4],[5,6]]
nd=np.array(c)
df=pd.DataFrame(d.tolist())
np.ravel(df)

array([1, 2, 3, 4, 5, 6], dtype=int64)

In [38]:
x_df.index.levels[0]
date_list=['2010/11/12','2020/11/12','2030/11/12']
a=pd.to_datetime(date_list)
b=pd.date_range(date_list[0],freq='M',periods=10)
c=pd.Series(np.random.randn(len(b)),index=b)
#print(c)
c['2010']
pd.__version__
df2 = x_df.copy()
df2.where(df2 > 0, df2['amount'], axis=0)

open      high       low     close    volume    amount
date       code                                                              
2018-03-20 600351  0.207361  0.758139  0.109008  0.699127  0.994348  0.977394
           601318  2.934789  3.224730  2.828389  3.198130  0.274366  0.274366
2018-03-21 600351  0.659786  0.758139  0.325385  0.345056 -0.096707 -0.096707
           601318  3.892392  3.895052  2.916169  3.126310  2.174753  2.868297
2018-03-22 600351  0.443409  0.561433  0.109008  0.168020 -0.288785 -0.288785
           601318  3.043850  3.219410  2.557068  2.876269  0.700123  1.100998
2018-03-23 600351  0.198820  0.198820  0.198820  0.198820  0.281905  0.198820
           601318  2.110187  2.315007  1.955906  2.189987  2.553938  2.943906
2018-03-26 600351 -0.627984 -0.627984 -0.627984 -0.627984 -0.627984 -0.627984
           601318  1.897386  2.102207  1.306864  1.652665  2.217601  2.469021
2018-03-27 600351 -0.115262 -0.115262 -0.115262 -0.115262 -0.115262 -0.115262
           601318  1.998467  2.019747  1.445185  1.620745  1.135087  1.355871
2018-03-28 600351 -0.775359 -0.775359 -0.775359 -0.775359 -0.775359 -0.775359
           601318  1.312184  1.493065  0.748263  0.886583  2.343929  2.440145
2018-03-29 600351 -0.897664 -0.897664 -0.897664 -0.897664 -0.897664 -0.897664
           601318  0.891903  1.386665  0.383842  1.149924  2.476466  2.503877
2018-03-30 600351 -0.431539 -0.431539 -0.431539 -0.431539 -0.431539 -0.431539
           601318  1.048844  1.118004  0.726983  0.862643  0.506973  0.585539
2018-04-02 600351 -0.424911 -0.424911 -0.424911 -0.424911 -0.424911 -0.424911
           601318  0.580682  1.112684  0.572702  0.689743  0.391243  0.462880
2018-04-03 600351 -0.752184 -0.752184 -0.752184 -0.752184 -0.752184 -0.752184
           601318  0.500882  0.766883  0.471622  0.649842 -0.185540 -0.185540
2018-04-04 600351  0.161404  0.187691  0.161404  0.161404  0.211036  0.161404
           601318  0.753583  0.934463  0.508862  0.660482  0.067389  0.067389
2018-04-09 600351 -0.238275  0.148349 -0.238275  0.069667 -0.238275 -0.238275
           601318  0.777523  1.043524  0.527482  1.035544  0.107440  0.193896
2018-04-10 600351  0.069667  0.404068  0.425109  0.384397  0.464326  0.425109
           601318  0.963723  1.684586  0.883923  1.644685  0.707302  0.855898
2018-04-11 600351  0.384397  0.463080  0.109008  0.128679  0.071021  0.064086
           601318  1.657985  1.729806  1.439865  1.490405  0.122618  0.296618
...                     ...       ...       ...       ...       ...       ...
2018-08-03 600351  0.325385  0.620445  0.522314  0.522314  0.576776  0.522314
           601318 -1.034098 -1.034098 -1.034098 -1.034098 -1.034098 -1.034098
2018-08-06 600351  0.027718  0.027718  0.027718  0.027718  0.108404  0.027718
           601318 -0.876715 -0.876715 -0.876715 -0.876715 -0.876715 -0.876715
2018-08-07 600351 -0.461055 -0.461055 -0.461055 -0.461055 -0.461055 -0.461055
           601318 -0.461996 -0.461996 -0.461996 -0.461996 -0.461996 -0.461996
2018-08-08 600351 -0.828095 -0.828095 -0.828095 -0.828095 -0.828095 -0.828095
           601318 -0.882204 -0.882204 -0.882204 -0.882204 -0.882204 -0.882204
2018-08-09 600351 -0.274105 -0.274105 -0.274105 -0.274105 -0.274105 -0.274105
           601318 -0.216561 -0.216561 -0.216561 -0.216561 -0.216561 -0.216561
2018-08-10 600351 -0.500163  0.207361 -0.500163  0.030326 -0.500163 -0.500163
           601318 -1.034304 -1.034304 -1.034304 -1.034304 -1.034304 -1.034304
2018-08-13 600351 -0.748835  0.128679 -0.748835  0.109008 -0.748835 -0.748835
           601318 -0.755155 -0.755155 -0.755155 -0.755155 -0.755155 -0.755155
2018-08-14 600351  0.030326  0.443409 -0.761194  0.168020 -0.761194 -0.761194
           601318 -1.170182 -1.170182 -1.170182 -1.170182 -1.170182 -1.170182
2018-08-15 600351  0.207361  0.266373 -0.550862 -0.550862 -0.550862 -0.550862
           601318 -0.354293 -0.354293 -0.354293 -0.354293 -0.354293 -0.354293
2018-08-16 600351 -1.021321 -1.021321 -1.0213

In [123]:
#x_df[['open','close']].plot()
x=x_df.copy()
#x.pivot(index='code', columns='close')
x.index.codes[1]
x['code']=x.index.codes[1]
a=x.iloc[0,:]
#a.iloc[0]=5
aaa=x_df.loc[(slice(None),'600351'),:].iloc[9:12,:]
bbb=x_df.loc[(slice(None),'600351'),:].iloc[0:5,:]

index = pd.MultiIndex.from_tuples([], names=['date', 'code'])

#print(df_empty)
#print((bbb.index))
df_empty = pd.DataFrame(columns=x_df.columns,index=index)
df_empty=pd.concat([df_empty,bbb],axis=0)
df_empty=pd.concat([df_empty,aaa],axis=0)
#df_empty[x_df.columns]=aaa
#df_empty[x_df.columns]=x_df.loc[(slice(None),'600351'),:].iloc[6:8,:]

df_empty

#bbb.values.flatten()
#x.pivot(index='code', columns='code')df_empty[x_df.columns]=bbb


,,open,high,low,close,volume,amount
date,code,,,,,,
2018-03-20,600351,0.207361,0.758139,0.109008,0.699127,0.994348,0.977394
2018-03-21,600351,0.659786,0.758139,0.325385,0.345056,-0.132837,-0.096707
2018-03-22,600351,0.443409,0.561433,0.109008,0.168020,-0.308867,-0.288785
2018-03-23,600351,-0.146710,-0.028686,-0.481111,-0.382758,0.281905,0.198820
2018-03-26,600351,-0.559794,-0.127040,-0.618806,-0.166381,-0.628904,-0.627984
2018-04-02,600351,-0.186051,-0.048357,-0.245063,-0.166381,-0.421018,-0.424911
2018-04-03,600351,-0.323746,-0.087698,-0.481111,-0.166381,-0.772648,-0.752184
2018-04-04,600351,-0.166381,0.187691,-0.284405,-0.068028,0.211036,0.161404
